In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import sklearn.model_selection
import sklearn.preprocessing

%matplotlib inline

/home/harrison/anaconda3/envs/mas-thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

In [2]:
df_raw = pd.read_pickle('data/processed.pkl')
df_raw = df_raw.set_index('DATE')  # Index for timeseries is datetime

df_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 592 entries, 1963-01-01 to 2012-04-01
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   MSACSR                      592 non-null    float64
 1   MSPUS                       592 non-null    float64
 2   S&P Comp.                   592 non-null    float64
 3   Dividend                    592 non-null    float64
 4   Earnings                    592 non-null    float64
 5   CPI                         592 non-null    float64
 6   Long Interest Rate GS10     592 non-null    float64
 7   Real Price                  592 non-null    float64
 8   Real Dividend               592 non-null    float64
 9   Real Total Return Price     592 non-null    float64
 10  Real Earnings               592 non-null    float64
 11  Real TR Scaled Earnings     592 non-null    float64
 12  CAPE                        592 non-null    float64
 13  Total Return CAP

### Data Interpolation

In [3]:
# Todo: Data interpolation if necessary

### Data Preprocessing

In [4]:
df = df_raw.copy()

# Set target variable
df['target'] = df_raw['logdiff_MSPUS']

# Remove extraneous columns
df = df.drop(['null_locations', 'null_count', 'Year', 'MSPUS'], axis='columns')

# Remove first row since null for differenced columns
df = df.drop(index=df.index[0], axis='index')

print(df.shape)
df.info()

(591, 21)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 591 entries, 1963-02-01 to 2012-04-01
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   MSACSR                      591 non-null    float64
 1   S&P Comp.                   591 non-null    float64
 2   Dividend                    591 non-null    float64
 3   Earnings                    591 non-null    float64
 4   CPI                         591 non-null    float64
 5   Long Interest Rate GS10     591 non-null    float64
 6   Real Price                  591 non-null    float64
 7   Real Dividend               591 non-null    float64
 8   Real Total Return Price     591 non-null    float64
 9   Real Earnings               591 non-null    float64
 10  Real TR Scaled Earnings     591 non-null    float64
 11  CAPE                        591 non-null    float64
 12  Total Return CAPE           591 non-null    float64
 13  Excess

In [5]:
# Split into input and target variables

X = df.drop('target', axis='columns')
y = df['target'].values
y = y.reshape(-1, 1)  # Put target variable into column vector shape

print(X.shape)
print(y.shape)

(591, 20)
(591, 1)


In [6]:
# Scale data

standard_scaler = sklearn.preprocessing.StandardScaler()  # Scales to mean of 0 and var of 1
minmax_scaler = sklearn.preprocessing.MinMaxScaler()  # Scale to between 0 and 1

X_scaled = standard_scaler.fit_transform(X)  
y_scaled = minmax_scaler.fit_transform(y)    

print("X mean:", X_scaled.mean())
print("X var:", X_scaled.var())
print("y min:", y_scaled.min())
print("y max:", y_scaled.max())

X mean: -6.462211852302096e-16
X var: 1.0
y min: 0.0
y max: 1.0


In [7]:
# NOTE: These values are for monthly data
#       Change these if interpolating!
IN_SEQ_LENGTH = 24
OUT_SEQ_LENGTH = 12

def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):
    X = []
    y = []
    for i in range(len(input_sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix > len(input_sequences):
            break
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)
        
    return np.array(X), np.array(y)

X_seq, y_seq = split_sequences(X_scaled, y_scaled, IN_SEQ_LENGTH, OUT_SEQ_LENGTH)

# Make 
print(X_seq.shape, y_seq.shape)

assert y_seq[0].all() == y_scaled[IN_SEQ_LENGTH-1:IN_SEQ_LENGTH-1+OUT_SEQ_LENGTH].squeeze(1).all()

(557, 24, 20) (557, 12)


In [8]:
TEST_SIZE = 0.05

# Split data into train and test sets
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_seq, y_seq, test_size=TEST_SIZE)

# Convert datasets to nodes in computational graph
X_train_tensors = torch.Tensor(X_train)
y_train_tensors = torch.Tensor(y_train)
X_val_tensors = torch.Tensor(X_val)
y_val_tensors = torch.Tensor(y_val)

# Convert to sequential data for pytorch
X_train_tensors = torch.reshape(X_train_tensors, 
                                (X_train_tensors.shape[0], IN_SEQ_LENGTH, X_train_tensors.shape[2])
                               )
X_val_tensors = torch.reshape(X_val_tensors, (X_val_tensors.shape[0], IN_SEQ_LENGTH, X_val_tensors.shape[2]))


print(X_train_tensors.shape)
print(X_val_tensors.shape)

torch.Size([529, 24, 20])
torch.Size([28, 24, 20])


In [9]:
# Sanity Check data
X_check, y_check = split_sequences(X, y.reshape(-1, 1), IN_SEQ_LENGTH, OUT_SEQ_LENGTH)
print(X_check[-1][0:4])

start_ix = IN_SEQ_LENGTH + OUT_SEQ_LENGTH

#print(X.iloc[-start_ix + 1: -start_ix + 5])

[[ 8.50000000e+00  9.26120000e+02  2.55900000e+01  7.51000000e+00
   2.15690000e+02  3.72000000e+00  1.24665000e+03  3.44500000e+01
   6.26478280e+05  1.01100000e+01  5.08018000e+03  1.63800000e+01
   1.80400000e+01  5.02000000e+00  1.02000000e+00  4.16700000e+01
   1.23700000e+01  2.17000000e+00  1.02100000e+01 -1.01103802e-02]
 [ 7.90000000e+00  9.35820000e+02  2.50300000e+01  9.19000000e+00
   2.15350000e+02  3.56000000e+00  1.26171000e+03  3.37400000e+01
   6.35458250e+05  1.23900000e+01  6.23810000e+03  1.66900000e+01
   1.84100000e+01  5.02000000e+00  1.00000000e+00  4.24200000e+01
   1.26200000e+01  2.38000000e+00  1.02400000e+01 -1.02136450e-02]
 [ 7.50000000e+00  1.00973000e+03  2.44600000e+01  1.08600000e+01
   2.15830000e+02  3.59000000e+00  1.35831000e+03  3.29100000e+01
   6.85492860e+05  1.46100000e+01  7.37498000e+03  1.80900000e+01
   1.99800000e+01  4.53000000e+00  1.02000000e+00  4.23500000e+01
   1.14100000e+01  2.41000000e+00  9.00000000e+00  7.28403066e-03]
 [ 7.80

### Model

In [10]:
# Define model

class LSTM(torch.nn.Module):
    def __init__(self, n_classes, n_inputs, n_hidden, n_layers):
        super().__init__()
        self.n_classes = n_classes  # Output size
        self.n_inputs = n_inputs  # Input size
        self.n_hidden = n_hidden
        self.n_layers = n_layers  # Number of reccurrent layers
        
        self.lstm = torch.nn.LSTM(input_size=n_inputs, 
                                  hidden_size=n_hidden,
                                  num_layers=n_layers,
                                  batch_first=True,  # Input/Output layers are of the form (batch, seq, feature)
                                  dropout=0.2  # Dropout helps to avoid over-fitting/improves robustness
                                 )
        self.fc1 = torch.nn.Linear(n_hidden, 128)  # Fully connected layer 1
        self.fc2 = torch.nn.Linear(128, n_classes)  # Fully connected layer 2
        self.relu = torch.nn.ReLU()  # Activation layer
        
    def forward(self, X):
        # Initialize hidden state
        h_0 = torch.Tensor(torch.zeros(self.n_layers, X.size(0), self.n_hidden))
        # Initialize cell state
        c_0 = torch.Tensor(torch.zeros(self.n_layers, X.size(0), self.n_hidden))
        
        # Forward propogate inputs
        output, (h_n, c_n) = self.lstm(X, (h_0, c_0))
        h_n = h_n.view(-1, self.n_hidden)
        out = self.relu(h_n)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [11]:
# Define training loop

def training_loop(n_epochs, lstm, optimizer, loss_fn, X_train, y_train, X_test, y_test):
    train_loss_list = []
    test_loss_list = []
    for epoch_i in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train)  # Forward pass
        optimizer.zero_grad()  # Calculate gradient, manually set to 0
        loss = loss_fn(outputs, y_train)
        loss.backward()  # Calculate loss
        optimizer.step()  # Backpropogate loss

        # Test set loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)
        if epoch_i % 100 == 0:
            print("Epoch {}: Train loss: {:.8f} Test loss: {:.15f}".format(epoch_i, 
                                                                          loss.item(),
                                                                          test_loss.item()
                                                                         )
                 )
        train_loss_list.append(loss.item())
        test_loss_list.append(test_loss.item())
    
    return train_loss_list, test_loss_list

In [ ]:
# Run model


# Define model parameters
N_EPOCHS = 1000
LEARNING_RATE = 0.001

INPUT_SIZE = X.shape[1]  # Number of features
HIDDEN_SIZE = 10  # Number of features in hidden state
N_LAYERS = 1 # Number of stacked lstm layers

N_CLASSES = OUT_SEQ_LENGTH  # Equal to how many timesteps in future we want to predict

lstm1 = LSTM(N_CLASSES,
            INPUT_SIZE,
            HIDDEN_SIZE,
            N_LAYERS
           )

# Define loss function
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm1.parameters(), lr=LEARNING_RATE)

# Run training loop
train_loss, test_loss = training_loop(n_epochs=N_EPOCHS,
              lstm=lstm1,
              optimizer=optimizer,
              loss_fn=loss_fn,
              X_train=X_train_tensors,
              y_train=y_train_tensors,
              X_test=X_val_tensors,
              y_test=y_val_tensors
             )

/home/harrison/anaconda3/envs/mas-thesis/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 0: Train loss: 0.33445346 Test loss: 0.340479075908661
Epoch 100: Train loss: 0.02252129 Test loss: 0.022162824869156


### Plot Results

In [ ]:
plt.plot(train_loss, label="Train Loss")
plt.plot(test_loss, label="Test Loss")
plt.title("Model Loss")
plt.legend()

plt.show()

In [ ]:
# Plot predictions vs actual

def inverse_pct_change(series, y_0):
    cumulative_sum = np.array(series).cumsum()
    
    return y_0 * np.exp(cumulative_sum)


# Get data into the right format
X_tensors = torch.Tensor(X_seq)
y_tensors = torch.Tensor(y_seq)

X_tensors = torch.reshape(X_tensors, (X_tensors.shape[0], IN_SEQ_LENGTH, X_tensors.shape[2]))

# Get predictions and actual
predictions_scaled = lstm1(X_tensors).data.numpy()  # Convert to numpy for plotting
actual_scaled = y_tensors.data.numpy()

# Reverse scaling transformations
predictions_seq = minmax_scaler.inverse_transform(predictions_scaled)  # y values used minmax scaler earlier
actual_seq = minmax_scaler.inverse_transform(actual_scaled)

# Reverse sequence transformations
predictions = [predictions_seq[i][0] for i in range(len(predictions_seq))]
actual = [actual_seq[i][0] for i in range(len(actual_seq))]

# Reverse log difference transformations
first_target = 17800  # First MSPUS value before log differencing
predictions_final = inverse_pct_change(predictions, first_target)
actual_final = inverse_pct_change(actual, first_target)

# Plot
plt.figure(figsize=(15, 8))


train_test_cutoff = round((1-TEST_SIZE) * X_tensors.shape[0])  # Index where test data starts
plt.axvline(x=train_test_cutoff, c='red', linestyle='--')

plt.plot(actual_final, label='Actual')
plt.plot(predictions_final, label='Predicted')
plt.legend()
plt.show()

In [ ]:
# Predict future based only on last in sequence length

# Get last MSPUS value before start of predictions
first_target = df_raw['MSPUS'].iloc[-(OUT_SEQ_LENGTH+1)]  

# Predictions
test_predictions = lstm1(X_val_tensors[-1].unsqueeze(0)).detach().numpy()  # Get the last sample
test_predictions = minmax_scaler.inverse_transform(test_predictions)  # Reverse scaling transform
test_predictions = test_predictions[0].tolist()
test_predictions = inverse_pct_change(test_predictions, first_target)

# Actual
test_actual = y_val_tensors[-1].detach().numpy()  # Last actual y
test_actual = minmax_scaler.inverse_transform(test_actual.reshape(1, -1))
test_actual = test_actual[0].tolist()
test_actual = inverse_pct_change(test_actual, first_target)

plt.plot(test_actual, label="Actual")
plt.plot(test_predictions, label="Predicted")
plt.legend()
plt.show()